In [1]:
from sklearnex import patch_sklearn, unpatch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


# 处理文本数据

In [2]:
# 表示数据属性的两种类型的特征 : 连续特征 与 分类特征
# 前者用于描述数量 ,后者是固定列表中的元素

# 在许多应用中还可以见到第三种类型的特征 : 文本

# 文本数据通常被表示为由字符组成的字符串
# 文本数据的长度都不相同
# 这个特征显然与数值特征有很大不同 ,需要预先处理数据 ,然后才能对其应用机器学习算法

## 用字符串表示的数据类型

In [3]:
# 文本通常只是数据集中的字符串 ,但并非所有的字符串特征都应该被当作文本来处理
# 字符串特征有时可以表示分类变量
# 在查看数据之前 ,我们无法知道如何处理一个字符串特征

<span class="burk">可能会遇到四种类型的字符串数据</span>

* 分类数据
* 可以在语义上映射为类别的自由字符串
* 结构化字符串数据
* 文本数据

In [4]:
# 分类数据(categorical data)是来自固定列表的数据

# 可以通过观察来判断你的数据是不是分类数据(如果你看到了许多不同的字符串 ,那么不太可能是分类变量)
# 并通过计算数据集中的唯一值并绘制其出现次数的直方图来验证你的判断
# 可能还希望检查每个变量是否实际对应于一个在应用中有意义的分类

In [5]:
# 从文本框中而得到的回答属于上述列表中的第二类 ,可以在语义上映射为类别的自由字符串
# (free strings that can be semantically mapped to categories)

# 如果你能够改变数据的收集方式 ,那么强烈建议 ,对于分类变量能够更好地表示的概念 ,不要使用手动输入值

In [6]:
# 通常来说 ,手动输入值不与固定的类别对应 ,但仍有一些内在的结构(structure)
# 比如 ,地址 ,人名 ,日期 ,电话号码 ,其他标识符

# 这种类型的字符串通常难以解析 ,其处理方法也强烈依赖于上下文和具体领域

In [7]:
# 最后一类字符串数据是自由格式的文本数据(text data) ,由短语或句子组成

# 所有这些集合包含的信息大多是由单词组成的句子

# 在文本分析的语境中
# 数据集通常被称为语料库(corpus)
# 每个由单个文本表示的数据点被称为文档(document)
# 这些属于来自于信息检索(imformation retrieval ,IR) 和 自然语言处理(natural language processing , NLP)
# 它们主要针对文本数据

<span class="burk">下载数据 :</span>
http://ai.stanford.edu/~amaas/data/sentiment/

In [8]:
!tree -d 2 /jinchanglong/aclImdb # window 系统会提示参数太多

参数太多 - 2


In [9]:
# pos文件夹包含所有正面的评论 ,每条评论都是一个单独的文本文件 ,neg文件夹与之类似
# scikit-learn 中由以各辅助函数可以加载用这种文件夹结构保存的文件 ,其中每个子文件夹对应一个标签
# 这个函数叫做load_files

In [10]:
from sklearn.datasets import load_files

reviews_train = load_files(r'C:\Users\jinchanglong\aclImdb\train')
# load_files返回一个Bunch对象 ,其中包含训练文本和训练标签
text_train , y_train = reviews_train.data , reviews_train.target
print('type of text_train :{}'.format(type(text_train)))
print('length of text_train :{}'.format(len(text_train)))
print('text_train[1]:\n{}'.format(text_train[1]))

type of text_train :<class 'list'>
length of text_train :75000
text_train[1]:
b"Amount of disappointment I am getting these days seeing movies like Partner, Jhoom Barabar and now, Heyy Babyy is gonna end my habit of seeing first day shows.<br /><br />The movie is an utter disappointment because it had the potential to become a laugh riot only if the d\xc3\xa9butant director, Sajid Khan hadn't tried too many things. Only saving grace in the movie were the last thirty minutes, which were seriously funny elsewhere the movie fails miserably. First half was desperately been tried to look funny but wasn't. Next 45 minutes were emotional and looked totally artificial and illogical.<br /><br />OK, when you are out for a movie like this you don't expect much logic but all the flaws tend to appear when you don't enjoy the movie and thats the case with Heyy Babyy. Acting is good but thats not enough to keep one interested.<br /><br />For the positives, you can take hot actresses, last 30 minutes,

In [11]:
# 可以看到 ,text_train 是一个长度为25000的列表 ,其中每个元素是包含一条评论的字符串
# 打印出索引编号为1的评论
# 还可以看到 ,评论中包含一些HTML换行符(<br />)
# 虽然这些符号不太可能对机器学习模型产生很大影响 ,但最好在继续下一步之前清洗数据并删除这种格式

In [12]:
text_train = [doc.replace(b'<br />',b' ') for doc in text_train]

In [13]:
# text_train 的元素类型与所使用的Python版本有关
# 在Python 3中 ,它们是bytes类型 ,是表示字符串数据的二进制编码
# 在Python 2中 ,test_train 包含的是字符串

In [14]:
# 收集数据集时保持正类和反类的平衡 ,这样所有正面字符串和负面字符串的数量相等:

In [15]:
import numpy as np

print('Sample per class (training) :{}'.format(np.bincount(y_train)))

Sample per class (training) :[12500 12500 50000]


In [16]:
# 用同样的方式加载测试数据集

reviews_test = load_files(r'C:\Users\jinchanglong\aclImdb\test')
text_test , y_test = reviews_test.data , reviews_test.target 
print('Number of documents in datasets :{}'.format(len(text_test)))
print('Sample per class (test):{}'.format(np.bincount(y_test)))
text_test = [doc.replace(b'<br />' ,b' ') for doc in text_test]

Number of documents in datasets :25000
Sample per class (test):[12500 12500]


In [17]:
# 我们要解决的任务如下 :
# 给定一条评论 ,希望根据该评论的文本内容对其分配一个'正面的'或'负面的'标签
# 这是一项标准的二分类任务
# 但是 ,文本数据并不是机器学习模型可以处理的格式
# 我们需要将文本的字符串表示转换为数值表示 ,从而可以对其应用机器学习算法

## 将文本数据表示为词袋

In [18]:
# 用于机器学习的文本表示有一种最简单的方法 ,也是最有效且最常用的方法 ,就是使用词袋(bag-of-words)表示
# 使用这种表示方式时 ,舍弃了输入文本中的大部分结构 ,如章节 ,段落 ,句子和格式 ,只计算语料库中每个单词在每个文本中出现频次
# 舍弃结构并仅计算单词出现次数 ,这会让脑海中出现将文本表示为'袋'的画面

<span class="burk">对于文档语料库 ,计算词袋表示包括以下三个步骤</span>

*分词 (tokenization). 将每个文档划分为出现在其中的单词[称为词例(token)],比如按空格和标点划分<br>
*构建词表(vocalulary building). 收集一个词表,里面包含出现在任意文档中的所有词 ,并对它们进行编号(比如按字母顺序排序)<br>
*编码(encoding). 对于每个文档 ,计算词表中每个单词在该文档中出现频次

In [19]:
# 其输出时包含每个文档中单词计数的一个向量
# 对于词表中每个单词 ,都有他在每个文档中的出现次数
# 也就是说 ,整个数据集中的每个唯一单词都对应于这种数值表示的一个特征
# 请注意 ,原始字符串中的单词顺序与词袋特征表示完全无关

### 将词袋应用于玩具数据集

In [20]:
# 词袋表示时在CountVectorizer中实现的 ,它是一个变换器(transformer)

In [21]:
# 首先将他应用与一个包含两个样本的玩具数据集 ,来看一下它的工作原理

bards_words = ['The foll doth think he is wise,','but the wise man knows himself to be a fool']

# 导入 CountVectorizer并将其实例化 ,然后对玩具数据进行拟合
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(bards_words)

# 拟合CountVectorizer包括训练数据的分词与词表的构建 ,可以通过vocabulary_属性来访问词表
print('Vocabulary size :{}'.format(len(vect.vocabulary_)))
print('\n')
print('Vocabulary content:\n {}'.format(vect.vocabulary_))
print('\n')
# 可以调用transform方法来创建训练数据的词袋表示:
bag_of_words = vect.transform(bards_words)
print('bag of words :{}'.format(repr(bag_of_words)))

Vocabulary size :14


Vocabulary content:
 {'the': 10, 'foll': 3, 'doth': 2, 'think': 11, 'he': 5, 'is': 7, 'wise': 13, 'but': 1, 'man': 9, 'knows': 8, 'himself': 6, 'to': 12, 'be': 0, 'fool': 4}


bag of words :<2x14 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>


In [22]:
# 词袋表示保存在一个SciPy稀疏矩阵中 ,这种数据格式只保存非零元素
# 这里使用稀疏矩阵 ,是因为大多数文档都只包含词表中的一小部分单词
# 也就是说 ,特征数组中的大部分元素都为0

# 保存所有0的代价很高 ,也浪费内存
# 想要查看稀疏矩阵的实际内容 ,可以使用toarray方法将其转换为'密集的'Numpy数组(保存所有0元素)

In [23]:
print('Dense representation of bag_of_words:\n{}'.format(bag_of_words.toarray()))

Dense representation of bag_of_words:
[[0 0 1 1 0 1 0 1 0 0 1 1 0 1]
 [1 1 0 0 1 0 1 0 1 1 1 0 1 1]]


In [24]:
# 可以看到 ,每个单词的计数都是0或1
# bards_words中的两个字符串都没有包含相同的单词

### 将词袋应用于电影评论

In [25]:
# 前面将IMDb评论的训练数据和测试数据加载为字符串列表(text_train 和text_test) 

vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)
print('X_train :\n{}'.format(repr(X_train)))

X_train :
<75000x124255 sparse matrix of type '<class 'numpy.int64'>'
	with 10315542 stored elements in Compressed Sparse Row format>


In [26]:
# 数据同样被保存为SciPy系数矩阵
# 访问词表的另一种方法时使用向量器(vertorizer)的get_feature_name方法 
# 它将返回一个列表 ,每个元素对应于一个特征

In [27]:
feature_names = vect.get_feature_names()
print('Number of features:{}'.format(len(feature_names)))
print('\n')
print('First 20 feature :\n{}'.format(feature_names[:20]))
print('\n')
print('Features 20010 to 20030:\n{}'.format(feature_names[20010:20030]))
print('\n')
print('Every 2000th feature:\n{}'.format(feature_names[::2000]))

Number of features:124255


First 20 feature :
['00', '000', '0000', '0000000000000000000000000000000001', '0000000000001', '000000001', '000000003', '00000001', '000001745', '00001', '0001', '00015', '0002', '0007', '00083', '000ft', '000s', '000th', '001', '002']


Features 20010 to 20030:
['cheapen', 'cheapened', 'cheapening', 'cheapens', 'cheaper', 'cheapest', 'cheapie', 'cheapies', 'cheapjack', 'cheaply', 'cheapness', 'cheapo', 'cheapozoid', 'cheapquels', 'cheapskate', 'cheapskates', 'cheapy', 'chearator', 'cheat', 'cheata']


Every 2000th feature:
['00', '_require_', 'aideed', 'announcement', 'asteroid', 'banquière', 'besieged', 'bollwood', 'btvs', 'carboni', 'chcialbym', 'clotheth', 'consecration', 'cringeful', 'deadness', 'devagan', 'doberman', 'duvall', 'endocrine', 'existent', 'fetiches', 'formatted', 'garard', 'godlie', 'gumshoe', 'heathen', 'honoré', 'immatured', 'interested', 'jewelry', 'kerchner', 'köln', 'leydon', 'lulu', 'mardjono', 'meistersinger', 'misspells', 'mumble

In [28]:
# 从无意义的'单词'中跳出有意义的有时很困难
# 进一步观察这个词表 ,发现许多以'dra'开头的英语单词
# 可能注意到了 ,对于'draught' ,'drawback' ,'drawer' ,其单数和复数形式都包含在词表中 ,并且作为不同的单词
# 这些单词具有密切相关的语义 ,将它们作为不同的单词进行计数(对应与不同的特征)可能不太合适

In [29]:
# 在尝试改进特征提取之前 ,先通过实际构建一个分类器来得到性能的量化度量
# 将训练标签保存在y_train中 ,训练数据的词袋表示保存在X_train中 ,因此可以在这个数据上训练一个分类器
# 对于这样的高维稀疏数据 ,类似LogisticRegression的线性模型通常效果最好

In [30]:
# 首先使用交叉验证对LogisticRegression 进行评估

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
scores = cross_val_score(LogisticRegression(max_iter = 3000) , X_train , y_train , cv = 5)
print('Mean cross-validation accuracy :{:.2f}'.format(np.mean(scores)))

C:\ProgramData\Anaconda3\lib\site-packages\daal4py\sklearn\linear_model\logistic_path.py:548: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\daal4py\sklearn\linear_model\logistic_path.py:548: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

Mean cross-validation accuracy :0.69


In [31]:
# LogisticRegression有一个正则化参数C ,可以通过交叉验证来调节它

In [32]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C':[0.001,0.01,0.1,1,10]}
grid = GridSearchCV(LogisticRegression(max_iter = 3000) ,param_grid , cv = 5)
grid.fit(X_train , y_train)
print('Best cross-validation score:{:.2f}'.format(grid.best_score_))
print('Best parameters:' , grid.best_params_)

C:\ProgramData\Anaconda3\lib\site-packages\daal4py\sklearn\linear_model\logistic_path.py:548: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\daal4py\sklearn\linear_model\logistic_path.py:548: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

Best cross-validation score:0.71
Best parameters: {'C': 0.1}


In [33]:
# 使用C = 0.1 得到的交叉验证分数是89%
# 现在 ,我们可以在测试集上评估这个参数设置的泛化性能

X_test = vect.transform(text_test)
print('{:.2f}'.format(grid.score(X_test , y_test)))

0.18


In [34]:
# 观察是否可以改进单词提取
# CountVectorizer 使用正则表达式提取词例
# 默认使用的正则表达式是'\b\w\w+\b'
# 它的含义是找到所有包含至少两个字母或数字(\w)且被词边界(\b)分割的字符序列
# 它不会匹配只有一个字母的单词 ,还会将类似'doesn't'或'bit.ly'之类的缩写分开 ,但它会将'h8ter'匹配为一个单词
# 然后 ,CountVectorizer 将所有单词转换为小写字母

In [35]:
# 得到了许多不包含信息量的特征(比如数字)
# 减少这种特征的一种方法是 ,仅使用至少在2个文档(或者至少5个,等等)中出现过的词例
# 仅在一个文档中出现的词例不太可能出现在测试集中 ,因此没什么用
# 可以用min_df参数来设置词例至少需要在多少个文档中出现过

In [36]:
vect = CountVectorizer(min_df = 5).fit(text_train)
X_train = vect.transform(text_train)
print('X_train with min_df:{}'.format(repr(X_train)))

X_train with min_df:<75000x44532 sparse matrix of type '<class 'numpy.int64'>'
	with 10191240 stored elements in Compressed Sparse Row format>


In [37]:
# 正如上面的输出所示 - 只有原始特征的三分之一左右

feature_names = vect.get_feature_names()

print('First 50 features :\n{}'.format(feature_names[:50]))
print('Features 20010 to 20030:\n{}'.format(feature_names[20010:20030]))
print('Every 700th feature:\n{}'.format(feature_names[::700]))

First 50 features :
['00', '000', '001', '007', '00am', '00pm', '00s', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '100', '1000', '1001', '100k', '100th', '100x', '101', '101st', '102', '103', '104', '105', '106', '107', '108', '109', '10am', '10pm', '10s', '10th', '10x', '11', '110', '1100', '110th', '111', '112', '1138', '115', '116', '117', '11pm', '11th']
Features 20010 to 20030:
['inert', 'inertia', 'inescapable', 'inescapably', 'inevitability', 'inevitable', 'inevitably', 'inexcusable', 'inexcusably', 'inexhaustible', 'inexistent', 'inexorable', 'inexorably', 'inexpensive', 'inexperience', 'inexperienced', 'inexplicable', 'inexplicably', 'inexpressive', 'inextricably']
Every 700th feature:
['00', 'accountability', 'alienate', 'appetite', 'austen', 'battleground', 'bitten', 'bowel', 'burton', 'cat', 'choreographing', 'collide', 'constipation', 'creatively', 'dashes', 'descended', 'dishing', 'dramatist', 'ejaculation', 'epitomize', 'extinguished', 'figment', 'forgot

In [38]:
# 数字的个数明显少了 ,有些生僻词或拼写错误似乎也都消失了
# 再次运行网格搜索来看一下模型的性能如何

grid = GridSearchCV(LogisticRegression(max_iter = 3000) , param_grid ,cv = 5)
grid.fit(X_train , y_train)
print('Best cross-validation score:{:.2f}'.format(grid.best_score_))

C:\ProgramData\Anaconda3\lib\site-packages\daal4py\sklearn\linear_model\logistic_path.py:548: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\daal4py\sklearn\linear_model\logistic_path.py:548: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

KeyboardInterrupt: 

In [ ]:
# 并没有改进模型 ,但减少要处理的特征数量可以加速处理过程 ,舍弃无用的特征也可能提高模型的可解释性

In [ ]:
# 如果一个文档中包含训练数据没有包含的单词 ,并对其调用CountVectorizer的transform方法 ,那么这些单词将被忽略
# 因为它们没有包含在字典中

# 这对分类来说不是一个问题 ,因为从不在训练数据中的单词中需得不到任何内容
# 但对于某些应用而言(比如垃圾邮件检测) ,添加一个特征来表示特定文档中由多少个所谓'词表外'单词可能会有所帮助
# 为了实现这一点 ,需要设置min_df ,否则这个特征在训练期间永远不会被用到

## 停用词

In [ ]:
# 删除没有信息量的单词还有一种方法 ,就是舍弃哪些出现次数太多以至于没有信息量的单词

# 由两种主要方法:
# 使用特定语言的停用词(stopword)列表
# 或者舍弃那些出现过于频繁的单词

# scikit-learn 的feature_extraction.text 模块中提供了英语停用词的内置列表

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
print('Number of stop words:{}'.format(len(ENGLISH_STOP_WORDS)))
print('Every 10th stopwords :\n{}'.format(list(ENGLISH_STOP_WORDS)[::10]))

In [ ]:
# 显然 ,删除上述列表中的停用词只能是特征数量减少318个(即上述列表的长度) ,但可能会提高性能

In [ ]:
# 指定stop_words = 'english'将使用内置列表
# 我们也可以扩展这个列表并传入我们自己的列表
vect = CountVectorizer(min_df = 5 , stop_words = 'english').fit(text_train)
X_train = vect.transform(text_train)
print('X_train with stop words:\n{}'.format(repr(X_train)))

In [ ]:
# 现在数据集中的特征数量减少了305个 ,说明大部分停用词(但不是所有)都出现了
# 再次运行网格搜索

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(LogisticRegression(max_iter = 3000) ,param_grid , cv = 5)
grid.fit(X_train , y_train)
print('Best cross-validation score:{:.2f}'.format(grid.best_score_))

In [ ]:
# 使用停用词后的网格搜索性能略有下降 - 不至于担心
# 但鉴于从27000多个特征中删除305个不太可能对性能或可解释性造成很大影响
# 所以使用这个列表似乎是不值得的

# 固定的列表主要对小型数据集很有帮助 ,这些数据集可能么有包含足够的信息 ,模型从数据本身无法判断除哪些单词是停用词

## 用tf-idf缩放数据

In [ ]:
# 另一种方法是按照预计的特征信息量大小来缩放特征 ,而不是舍弃那些认为不重要的特征
# 最常见的一种做法就是使用词频 -逆向文档频率(term frequency-inverse document frequency ,tf-idf)方法
# 这一方法对在某个特定文档中经常出现的术语给予很高的权重 ,但对在语料库的许多文档中都经常出现的术语基于的权重却不高
# 如果一个单词在某个特定文档中经常出现 ,但在许多文档中却不常出现 ,那么这个单词很可能是对文档内容的很好描述

# scikit-learn 在两个类中实现了df-idf方法:
# TfidfTransformer 和 TfidfVetorizer 
# 前者接受CountVectorizer生成的稀疏矩阵并将其变换 
# 后者接受文本数据并完成词袋特征提取与tf-idf变换

In [ ]:
# 单词w在文档d中的tf-idf分数在TfidTransformer类和TfidfVectorizer类中都有实现

tfidf(w,d) = tf log$\frac{N + 1}{N_w + 1}$ + 1

In [ ]:
# 其中N是训练集中的文档数量 ,Nw是训练集中出现单词w的文档数量 ,tf(词频)是单词w在查询文档d中出现的次数
# 两个类在计算tf-idf表示之后 ,都还应用了L2范数

# 换句话说 ,它们将每个文档的表示缩放到欧几里得范数为1
# 利用这种缩放方法 ,文档长度(单词数量)不会改变向量化表示

In [ ]:
# 由于tf-idf实际上利用了训练数据的统计学属性
# 管道确保网格搜索的结果有效

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(TfidfVectorizer(min_df = 5) , LogisticRegression())

param_grid = {'logisticregression__C':[0.001,0.01,0.1,1,10]}

grid = GridSearchCV(pipe , param_grid , cv = 5)
grid.fit(text_train , y_train)
print('Best cross-validation score:{:.2f}'.format(grid.best_score_))

In [ ]:
# 使用tf-idf代替仅统计词数对性能有所提高
# 还可以查看tf-idf找到的最重要的单词
# tf-idf缩放的目的是找到能够区分文档的单词
# 但它完全是一种无监督计数
# 因此 ,这里的'重要'不一定与我们感兴趣的'正面评论'和'负面评论'标签相关

In [ ]:
# 首先 ,从管道中提取TfidfVectorizer

vectorizer = grid.best_estimator_.named_steps['tfidfvectorizer']
# 变化训练数据集
X_train = vectorizer.transform(text_train)
# 找到数据集中每个特征的最大值
max_value = X_train.max(axis = 0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
# 获取特征名称
feature_names = np.array(vectorizer.get_feature_names())

print('Features with lowest tfidf:\n{}'.format(feature_names[sorted_by_tfidf[:20]])
print('Features with highest tfidf: \n{}'.format(feature_names[sorted_by_tfidf[-20:]])

In [ ]:
# tf-idf 较小的特征要么是在许多文档例都很常用 ,要么就是很少使用 ,且仅出现在非常长的文档中
# 有趣的是 ,许多tf-idf较大的特征实际上对应的是特定的演出或电影

In [ ]:
# 还可以找到逆向文档频率较低的单词 ,即出现次数很多 ,因此被认为不那么重要的单词
# 训练集的逆向文档频率值被保存在idf_属性中

sorted_by_idf = np.array(vectorizer.idf_)
print('Features with lowest idf:\n{}'.format(feature_names[sorted_by_idf[:100]]))

In [ ]:
# 这些词大多是英语中的停用词
# 因此根据tf-idf度量也属于'不太相关'的单词 ,尽管可能认为这些单词对情感分析任务非常重要

## 研究模型系数

In [ ]:
# Logistic回归模型从数据中实际学到的内容
# 由于特征数量非常多 ,所以显然不能同时查看所有系数
# 但是 ,可以查看最大系数 ,并查看这些系数对应的单词
# 将使用基于tf-idf特征训练的最后一个模型

In [ ]:
# Logistic 回归模型中最大的25个系数与最小的25个系数 ,其高度表示每个系数的大小
import mglearn

mglearn.tools.visualize_coefficients(grid.best_estimator_.named_steps['logisticregression'].coef_ , 
                                     feature_names , n_top_features = 40)

In [ ]:
# 左侧的负系数属于模型找到的表示负面评论的单词 ,而右侧的正系数属于模型找到的表示正面评论的单词

## 多个单词的词袋(n元分词)

In [ ]:
# 使用词袋表示的主要缺点之一是完全舍弃了单词顺序

# 幸运的是 ,使用词袋表示时有一种获取上下文的方法 ,就是不仅考虑单一词例的计数 ,而却还考虑相邻的两个或三个词例的计数

# 两个词例被称为二元分词(bigram) ,三个词例被称为三元分词(trigram) ,更一般的词例序列被称为n元分词(n-gram)
# 可以通过改变CountVectorizer 或TfidfVectorizer 的ngram_range参数来改变作为特征的词例范围
# ngram_range 参数是一个元组 ,包含要考虑的词例序列的最小长度和最大长度

In [ ]:
print('bards_words :\n{}'.format(bards_words))

In [ ]:
# 默认情况下 ,为每个长度最小为1且最大为1的词例序列(或者换句话说 ,刚好1个词例)创建一个特征 - 单个词例也被称为一元分词(unigram)

In [ ]:
cv = CountVectorizer(ngram_range = (1,1)).fit(bards_words)
print('Vocabulary size:{}'.format(len(cv.vacabulary_)))
print('Vocabulary :\n{}'.format(cv.get_feature_names()))

In [ ]:
# 想要仅查看二元分词(即仅查看由两个相邻词例组成的序列) ,可以将ngram_range设置为(2,2)

In [ ]:
cv = CountVectorizer(ngram_range = (2,2)).fit(bards_words)
print('Vocabulary size:{}'.format(len(cv.vacabulary_)))
print('Vocabulary :\n{}'.format(cv.get_feature_names()))

In [ ]:
# 使用更长的词例序列通常会得到更多的特征 ,也会得到更具体的特征
# bards_words的两个短语中没有相同的二元分词

In [ ]:
print('Transformed data (dense):\n{}'.format(cv.transform(bards_words).transfor(toarray()))

In [ ]:
# 对于大多数应用而言 ,最小的词例数量应该是1 ,因为单个单词通常包含丰富的含义
# 在大多数情况下 ,添加二元分词会有所帮助
# 添加更长的序列(一直到五元分词)也可能由所帮助 ,但这会导致特征数量的大大增加 ,也可能会导致过拟合 ,因为其中包含许多非常具体的特征
# 原则上来说 ,二元分词的数量是一元分词数量的平方 ,三元分词的数量是一元分词数量的三次方 ,从而导致非常大的特征空间
# 在实践中 ,更高的n元分词在数据中的出现次数实际上更少 ,原因在于(英语)语言的结构 ,不过这个数字仍然很大

In [ ]:
# bards_words上使用一元分词 ,二元分词和三元分词

cv = CountVectorizer(ngram_range = (1,3)).fit(bards_words)
print('Vocabulary size:{}'.format(len(cv.vacabulary_)))
print('Vocabulary :\n{}'.format(cv.get_feature_names()))

In [ ]:
# 在电影评论数据上尝试使用TfidfVectorizer , 并利用网格搜索找出n元分词的最佳设置

pipe = make_pipeline(TfidfVectorizer(min_df = 5),LogisticRegression(max_iter = 5000))
# 运行网格搜索需要很长时间 ,因为网格相对较大 ,且包含三元分词
param_grid = {'logisticregression__C':['0.001,0.01,0.1,1,10,100'],
              'tfidfvectorizer__ngram_range':[(1,1),(1,2),(1,3)]}

grid = GridSearchCV(pipe , param_grid ,cv = 5)
grid.fit(text_train ,y_train)
print('Best cross-validation score:{:.2f}'.format(grid.best_score_))
print('Best parameters:\n{}'.format(grid.best_params_))

In [ ]:
# 从结果可以看出 ,添加了二元分词特征与三元分词特征之后 ,性能提高了一个百分点多一点
# 可以将交叉验证精度作为ngram_range 和 C参数的函数并用热图可视化

In [ ]:
# 从网格搜索中提取分数
scores = grid.cv_results_['mean_test_score'].reshape(-1,3).T

# 热图可视化
heatmap = mglearn.tools.heatmap(scores , xlabel = 'C' ,ylabel = 'ngram_range' ,cmap = 'viridis' , fmt = '%.3f' ,
                                xticklabels = param_grid['logisticregression__C'] , 
                                yticklabels = param_grid['tfidfvectorizer__ngram_range'])
plt.colorbar(heatmap)

In [ ]:
# 从热图中可以看出 ,使用二元分词对性能有很大提升 ,而添加三元分词对精度只有很小贡献
# 为了更好地理解模型是如何改进的 ,可以将最佳模型的重要系数可视化 ,其中包含一元分词 ,二元分词和三元分词

In [ ]:
# 提取特征名称与系数
vect = grid.best_estimator_.named_steps['tfidfvectorizer']
feature_names = np.array(vect.get_feature_names())
coef = grid.best_estimator_.named_steps['logisticregression'].coef_
mglearn.tools.visualize_coefficients(coef , feature_names , n_top_features = 40)

In [ ]:
# 找到三元分词特征
mask = np.array([len(feature.split(' ')) for feature in feature_names]) == 3
# 仅将三元分词特征可视化
mglearn.tools.visualize_coefficients(coef.ravel()[mask] , 
                                     feature_names[mask] , n_top_features = 40)

## 高级分词 ,词干提取与词形还原

In [ ]:
# CountVectorizer 和 TfidfVectorizer 中的特征提取相对简单 ,还有更为富在的方法
# 在更加复杂的文本处理应用中 ,通常需要改进的步骤是词袋模型的第一步 : 分词(tokenization)
# 这一步骤为特征提取定义了一个单词是如何构成的

In [ ]:
# 与名词的单复数形式一样 ,将不同的动词形式及相关单词视为不同的词例
# 这不利于构建具有良好泛化性能的模型

In [ ]:
# 这个问题可以通过用词干(word stem)表示每个单词来解决 ,这一方法设计找出[或合并(conflate)]所有具有相同词干的单词
# 如果使用基于规则的启发法来实现(比如删除常见的后缀) ,那么通常将其称为词干提取(stemming)
# 如果使用的是由一致单词形式组成的字典(明确的且经过人工验证的系统) ,并且考虑了单词在句子中的作用
# 那么这个过程被称为词形还原(lemmatization) ,单词的标准化形式被称为词源(lemma)
# 词干提取和磁性还原这两种处理方法都是标准化(normalization)的形式之一 ,标准化是指尝试提取一个单词的某种标准形式
# 标准化的另一个有趣的例子是拼写校正

In [52]:
import spacy
import nltk

# 加载spacy的英语模型
en_elp = spacy.load('en')
# 将nltk的Porter词干提取器实例化
stemmer = nltk.stem.PorterStemmer()

# 定义一个函数来对比spacy中的词性还原与nltk中的词干提取
def compare_normalization(doc):
    # 在spacy中对文档进行分词
    doc_spacy = en_elp(doc)
    # 打印出spacy找到的词源
    print('Lemmatization:')
    print([token.lema_ for token in doc_spacy])
    # 打印出porter词干提取器找到的词例
    print('Stemming')
    print('stemmer.stem(token.norm_.lower())' for token in doc_spacy)

OSError: [E941] Can't find model 'en'. It looks like you're trying to load a model from a shortcut, which is obsolete as of spaCy v3.0. To load the model, use its full name instead:

nlp = spacy.load("en_core_web_sm")

For more details on the available models, see the models directory: https://spacy.io/models. If you want to create a blank model, use spacy.blank: nlp = spacy.blank("en")

In [40]:
# 将用一个句子来比较词性还原与Porter词干提取器 ,以显示二者的一些区别

compare_normalization(u'Our meeting today was worse than yesterday,',
                      'I\'m scared of meeting the clients tomorrow.')

NameError: name 'compare_normalization' is not defined

In [1]:
# 词干提取总是局限于将单词讲话成词干  -> ('was' -> 'wa') -> ('worse' -> 'bad') 
# 另一个区别 : 利用词干提取 : 'meeting' -> 'meet'
# 利用词形还原 : (n)meeting -> meeting  (v)meeting -> meet

# 一般来说 ,词形还原是一个比词干提取更复杂的过程 ,但用于机器学习的词例标准化时通常可以给出比词干提取更好的结果

In [ ]:
# 虽然scikit-learn没有实现这两种形式的标准化
# 但 CountVectorizer 允许使用tokenizer参数来指定使用你自己的分词器将每个文档转换为词例列表
# 可以使用spacy的词形还原了创建一个可调用对象 ,它接受一个字符串并生成一个词元列表

In [ ]:
# 技术细节 : 我们希望使用由 CountVectorizer所使用的基于正则表达式的分词器
# 并仅使用spacy的词形还原
# 为此 ,我们将en_npl.tokenizer (spacy分词器)替换为基于正则表达式的分词

import re
# 在CountVectorizer中使用正则表达式
regexp = re.compile('(?u)\\b\\w\\w+\\b')

# 加载spacy语言模型 ,并保存旧的分词器
en_nlp = spacy.load('en')
old_tokenizer = en_nlp.tokenizer
# 将分词器替换为前面的正则表达式
en_nlp.tokenizer = lambda string:old_tokenizer.tokens_from_list(regexp.findall(string))

# 用spcay文档处理管道创建一个自定义分词器
# (现在使用我们自己的分词器)
def custom_tokenizer(document):
    doc_spacy = en_nlp(document , entity = False , parse = False)
    return [token.lemma_ for token in doc_spacy]

# 利用自定义分词器来定义一个计数向量器
lemma_vect = CountVectorizer(tokenizer = custom_tokenizer , min_df = 5)

In [ ]:
# 变换数据并检查词表的大小

In [ ]:
# 利用带词形还原的CountVectorizer 对 text_train进行变换
X_train_lemma = lemma_vect.fit_transform(text_train)
print('X_train_lemma:shape:{}'.format(X_train_lemma.shape))

# 标准的CountVectorizer , 以供参考
vect = CountVectorizer(min_df = 5).fit(text_train)
X_train = vect.transform(text_train)
print('X_train.shape:{}'.format(X_train.shape))

In [ ]:
# 词形还原可以被看作是一种正则化 ,因为它合并了某些特征
# 因此预计 ,数据集很小时词形还原对性能的提升最大
# 为了说明词形还原的作用 ,将使用StratifiedShuffleSplit做交叉验证 
# 仅使用1%的数据作为训练数据 ,其余数据作为测试数据

In [ ]:
# 仅使用1%的数据作为训练集来构建网格搜索
from sklearn.model_selection import StratifiedShuffleSplit

param_grid = {'C':[0.001,0.01,0.1,1,10]}
cv = StratifiedShuffleSplit(n_splits = 5 , test_size = 0.99 , train_size = 0.01 ,random_state = 0)
grid = GridSearchCV(LogisticRegression() ,param_grid ,cv = cv)
# 利用标准的CountVectorizer进行网格搜索
grid.fit(X_train , y_train)
print('Best cross-validation score''(standard CountVectorizer):{:.3f}'.format(grid.best_score_))
# 利用词形还原进行网格搜索
grid.fit(X_train_lemma , y_train)
print('Best cross-validation score''(lemmatization):{:.3f}'.format(grid.best_score_))

In [ ]:
# 在这个例子中 ,词形还原对性能由较小的提高
# 与许多特征提取技术一样 ,其结果因数据集的不同而不同
# 词形还原与词干提取有时有助于构建更好的模型(或至少是更简洁的模型)
# 所以建议 ,在特定任务中努力提升最后一点性能时可以尝试下这个技术

## 主题建模及文档聚类

In [ ]:
# 常用于文本数据的一种特殊技术时主题建模(topic modeling)
# 这是描述将每个文档分配给一个或多个主题的任务(通常是无监督的)的概括性术语
# 这方面一个很好的例子是新闻数据
# 它们可以被分为'政治','体育','金融'等主题
# 如果为每个文档分配一个主题 ,那么这是一个文档聚类任务

# 我们学到的每个成分对应于一个主题 ,文档表示中的成分系数告诉我们这个文档与该主题的相关性强弱
# 通常来说 ,人们在谈论主题建模时 ,他们指的是一种叫做隐含狄利克雷分布(Latent Dirichlet Allocation ,LDA)的特定分解方法

### 隐含狄利克雷分布

In [3]:
# 从直观上来看 ,LDA模型试图找出频繁共同出现的单词群组(即主题)
# LDA还要求 ,每个文档可以被理解为主题子集的'混合'

# 重要的是要理解 ,机器学习模型所谓的'主题'可能不是我们通常在日常对话中所说的主题
# 而是更类似于PCA 或 MNF 所提取的成分 ,他可能具有语义 ,也可能没有

# 即使LDA'主题'具有语义 ,他可能也不是我们通常所说的主题

In [ ]:
# 将LDA应用于电影评论数据集 ,来看一下它在实践中的效果
# 对于无监督的文本文档模型 ,通常最好删除非常常见的单词 ,否则他们可能会支配分析过程
# 将删除至少在15%的文档中出现过的单词 ,并在删除前15%之后 ,将词袋模型限定为最常见的10000个单词

In [ ]:
vect = CountVectorizer(max_features = 10000 , max_df = 0.15)
X = vect.fit_transform(text_train)

In [ ]:
# 将学习一个包含10个主题的主题模型 ,它包含的主题个数很少 ,可以查看所有主题
# 与NMF中的分量类似 ,主题没有内在的顺序 ,而改变主题数量将会改变所有主题
# 将使用'batch'学习方法 ,他比默认方法'online'稍慢 ,但通常会给出更好的结果
# 还将增大max_iter ,这样会得到更好的模型

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_topics = 10 , learning_method = 'batch',
                                max_iter = 25 , random_state = 0 )
# 在一个步骤中构建模型并变换数据
# 计算变换需要花时间 ,二者同时进行可以节省时间
document_topics = lda.fit_transform(X)

In [ ]:
# LatentDirichletAllocation 有一个 components_属性
# 其中保存了每个单词对每个主题的重要性
# components_的大小为(n_topics , n_words)

In [ ]:
lda.components_.shape

In [ ]:
# 为了更好地理解不同主题的含义 ,将查看每个主题中最重要的单词
# print_topics函数为这些特征提供了良好的格式

In [ ]:
# 对于每个主题(components_的一行) ,将特征排序(升序)
# [:,::-1]将行反转 ,使排序变为降序
sorting = np.argsort(lda.components_ , axis = 1)[:,::-1]
# 从向量器中获取特征名称
feature_names = np.array(vect.get_feature_names())

In [ ]:
# 打印出前10个主题
mglearn.tools.print_topics(topics = range(10) ,feature_names = feature_names ,
                           sorting = sorting , topics_per_chunk = 5 , n_words = 10)


In [ ]:
# 仅使用10个主题 ,每个主题都需要非常宽泛 ,才能共同覆盖我们的数据集中所有不同类型的评论

In [ ]:
# 接下来 ,将学习另一个模型
# 这次包含100个主题 ,将使的分析过程更加困难 ,但更可能使主题专门针对于某个有趣的数据子集

In [ ]:
lda100 = LatentDirichletAllocation(n_topics = 100 , learning_method = 'batch' , 
                                   max_iter = 25 ,random_state = 0)
document_topics100 = lda100.fit_transform(X)

# 查看所有100个主题可能有点困难 ,所以选取了一些有趣的而且有代表性的主题

In [ ]:
topics = np.array([7,16,24,25,28,36,37,45,51,53,54,63,89,97])

sorting = np.argsort(lda100.components_ , axis = 1)[:,::-1]
feature_names = np.array(vect.get_feature_names())
mglearn.tools.print_topics(topics = topics ,feature_names = feature_names , 
                           sorting = sorting ,topics_per_chunk = 7 ,n_words = 20)

In [ ]:
# 这次提取的主题似乎更加具体 ,不过很多都难以解读
# 如果想要利用发现的主题做出进一步的推断 ,那么查看分配给这些主题的文档
# 以验证通过查看每个主题排名最靠前的单词所得到的直觉

In [ ]:
# 按主题45 'music'进行排序
music = np.argsort(document_topics100[:,45])[::-1]
# 打印出这个主题最重要的前5个文档
for i in musicsic[:10]:
    # 显示前两个句子
    print(b'.'.join(text_train[i].split(b'.')[:2])+ b'.\n')

In [ ]:
# 查看主题还有一种有趣的方法 ,就是通过对所有评论的document_topics进行求和来查看每个主题所获得的整体权重
# 用最常见的两个单词为每个主题命名

In [ ]:
fig , ax = plt.subplots(1,2,figsize = (10,10))
topic_names = ['{:>2}'.format(i) + ' '.join(words) 
               for i ,words in enumerate(feature_names[sorting[:,:2]])]
# 两列的条形图
for col in [0,1]:
    start = col * 50
    end = (col + 1) * 50
    ax[col].barh(np.arange(50) , np.sum(document_topics100 , axis = 0)[start:end])
    ax[col].set_yticks(np.arange(50))
    ax[col].set_yticklabels(topic_names[start:end] , ha = 'left' ,va = 'top')
    ax[col].invert_yaxis()
    ax[col].set_xlim(0,2000)
    yax = ax[col].get_yaxis()
    yax.set_tick_params(pad = 130)
plt.tight_layout()

In [ ]:
# 除了几个不太具体的主题外 ,LDA似乎主要发现了两种主题
# 特定类型的主题与特定评分的主题
# 这是一个有趣的发现 ,因为大部分评论都由一些与电影相关的评论与一些证明或强调评分的评论组成

In [ ]:
# 在没有标签的情况下 ,想LDA这样的主题模型是理解大型文本语料库的有趣方法
# 不过LDA算法是随机的 ,改变random_state参数可能会得到完全不同的结果
# 虽然找到主题可能很有用 ,但对于从无监督模型中得出的任何结论都应该持保留态度
# 建议通过查看特定主题中的文档来验证你的直觉 
# LDA.transform方法生成的主题有时也可以用于监督学习的紧凑表示 
# 当训练样例很小时 ,这一方法特别有用

## 小结与展望

In [ ]:
# 正如机器学习中常见的情况 ,数据表示是NLP应用的关键 ,检查所提取的词例和n元分词有助于深入理解建模过程
# 在文本处理应用中 ,对于监督任务与无监督任务而言 ,通常都可以用有意义的方式对模型进行内省

In [ ]:
# 如前所述 ,CountVectorizer 类和 TFidfVectorizer 类仅实现了相对简单的文本处理方法
# 对于更高级的文本处理方法 ,推荐使用Python包spacy ,nltk 和 gensim

In [ ]:
# 文本处理方面都和神经网络有关 
# 第一进展是使用连续向量表示 ,也叫做词向量(word vector)或分布式词表示(distributed word representation)
# 它在word2vec库中实现

In [ ]:
# NLP还有另一个研究方向就是使用递归神经网络(recurrent neural network ,RNN)进行文本处理
# 与只能分配类别标签的分类模型相比 ,RNN是一种特别强大的神经网络 ,可以生成同样是文本的输出
# 能够生成文本作为输出 ,使得RNN非常适合自动翻译和摘要